In [1]:
import os
import zipfile
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, datasets

from collections import Counter

import openpifpaf
import cv2
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
from glob import glob
try:
    from PIL import Image
except ImportError:
    import Image
from time import time
from tqdm import tqdm, tqdm_notebook
import json

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization, LSTM, ConvLSTM2D, TimeDistributed, Conv2D, Conv1D, MaxPooling2D, MaxPooling1D, Attention
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from xgboost.sklearn import XGBClassifier
import pickle 
import warnings
warnings.filterwarnings("ignore")

In [2]:
tf.__version__

'2.6.0'

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [5]:
# skeletons_action_folders = glob('./datasets/Weizmann/heatmapsfeatures/*')
skeletons_action_folders = glob('./datasets/Weizmann/skeletons_croppeds/*')

In [5]:
dict_data = {'label': [], 'average_image': []}
for skeleton_folder in tqdm_notebook(skeletons_action_folders):
    
    current_action = skeleton_folder.split('\\')[1]
    
    actors_folders = glob(skeleton_folder.replace('\\', '/')+'/*')
    
    for actor_folder in actors_folders:
        
        dict_data['label'].append(current_action)
        
        images = glob(actor_folder.replace('\\', '/')+'/*')
        
        w,h=Image.open(images[0]).size
        N=len(images)

        # Create a numpy array of floats to store the average (assume RGB images)
        arr=np.zeros((h,w,3),np.float)

        # Build up average pixel intensities, casting each image as an array of floats
        for im in images:
            imarr=np.array(Image.open(im),dtype=np.float)
            arr=arr+imarr/N

        # Round values in array and cast as 8-bit integer
        arr=np.array(np.round(arr),dtype=np.uint8)
        
        dict_data['average_image'].append(arr)
        # Generate, save and preview final image
        out=Image.fromarray(arr,mode="RGB")
        out.save("Average.png")

  0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
dict_data['average_image'][0].shape

(100, 60, 3)

In [7]:
df = pd.DataFrame(dict_data)

# Logistic Regression

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['average_image'].values, df['label'].values, test_size=0.3, random_state=42)

In [9]:
X_train = [item.ravel() for item in X_train]

In [10]:
X_test = [item.ravel() for item in X_test]

In [11]:
clf = LogisticRegression()

In [12]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
y_pred = clf.predict(X_test)

In [14]:
y_pred[5]

'jump'

In [15]:
y_test[5]

'jump'

In [16]:
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

        bend       1.00      1.00      1.00         2
        jack       1.00      1.00      1.00         4
        jump       1.00      1.00      1.00         3
       pjump       1.00      1.00      1.00         4
         run       0.60      1.00      0.75         3
        side       0.33      1.00      0.50         1
        skip       0.50      0.50      0.50         2
        walk       1.00      0.33      0.50         6
       wave1       1.00      1.00      1.00         2
       wave2       1.00      1.00      1.00         1

    accuracy                           0.82        28
   macro avg       0.84      0.88      0.82        28
weighted avg       0.90      0.82      0.81        28

0.8214285714285714


In [8]:
loo = LeaveOneOut()

In [37]:
loo.get_n_splits(df['average_image'].values)

93

In [38]:
y_pred_all = []
y_true_all = []
for train_index, test_index in tqdm_notebook(loo.split(df['average_image'].values)):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = df['average_image'].values[train_index], df['average_image'].values[test_index]
    y_train, y_test = df['label'].values[train_index], df['label'].values[test_index]

    X_train = [item.ravel() for item in X_train]
    X_test = [item.ravel() for item in X_test]
    
    clf = LogisticRegression()
    
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    y_pred_all.append(y_pred[0])
    y_true_all.append(y_test[0])

0it [00:00, ?it/s]

In [39]:
print(classification_report(y_true_all, y_pred_all))

              precision    recall  f1-score   support

        bend       1.00      1.00      1.00         9
        jack       1.00      1.00      1.00         9
        jump       0.70      0.78      0.74         9
       pjump       0.90      1.00      0.95         9
         run       0.80      0.80      0.80        10
        side       1.00      0.89      0.94         9
        skip       0.56      0.50      0.53        10
        walk       0.83      1.00      0.91        10
       wave1       1.00      0.89      0.94         9
       wave2       1.00      0.89      0.94         9

    accuracy                           0.87        93
   macro avg       0.88      0.87      0.87        93
weighted avg       0.87      0.87      0.87        93



In [40]:
print(accuracy_score(y_true_all, y_pred_all))

0.8709677419354839


# CNN

In [17]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(100,60,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [18]:
optimizer = tf.keras.optimizers.Adam()
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [19]:
df['average_image'] = df['average_image'].apply(lambda x: np.asarray(x).astype(np.float32))

In [20]:
X_train, X_test, y_train, y_test = train_test_split(df['average_image'].values, df['label'].values, test_size=0.3, random_state=42)

In [21]:
X_train = X_train / 255.
X_test = X_test / 255.

In [22]:
# X_train = [np.reshape(item, (-1, 160,160, 3)) for item in X_train]

In [23]:
# X_test = [np.reshape(item, (-1, 160,160, 3)) for item in X_test]

In [24]:
dict_map = {}
for i, label in enumerate(df['label'].unique()):
    
    dict_map[label] = i

In [25]:
dict_map

{'bend': 0,
 'jack': 1,
 'jump': 2,
 'pjump': 3,
 'run': 4,
 'side': 5,
 'skip': 6,
 'walk': 7,
 'wave1': 8,
 'wave2': 9}

In [26]:
y_test = np.array([dict_map[y_t] for y_t in y_test])

In [27]:
y_train = np.array([dict_map[y_t] for y_t in y_train])

In [28]:
X_train = [np.asarray(x).astype(np.float32) for x in X_train]

In [29]:
X_test = [np.asarray(x).astype(np.float32) for x in X_test]

In [30]:
X_train = np.array(X_train)

In [31]:
X_test = np.array(X_test)

In [32]:
tf.config.run_functions_eagerly(True)

In [33]:
model.fit(X_train, y_train,
          epochs=50,
          validation_data=(X_test,y_test))

Epoch 1/50
3/3 [==============================] - 2s 143ms/step - loss: 2.2917 - accuracy: 0.1538 - val_loss: 2.4180 - val_accuracy: 0.0357
Epoch 2/50
3/3 [==============================] - 0s 79ms/step - loss: 2.2357 - accuracy: 0.1231 - val_loss: 2.5528 - val_accuracy: 0.0357
Epoch 3/50
3/3 [==============================] - 0s 81ms/step - loss: 2.2004 - accuracy: 0.1538 - val_loss: 2.5201 - val_accuracy: 0.0714
Epoch 4/50
3/3 [==============================] - 0s 76ms/step - loss: 2.0932 - accuracy: 0.2462 - val_loss: 2.6194 - val_accuracy: 0.0714
Epoch 5/50
3/3 [==============================] - 0s 80ms/step - loss: 2.0837 - accuracy: 0.2462 - val_loss: 2.5161 - val_accuracy: 0.0714
Epoch 6/50
3/3 [==============================] - 0s 83ms/step - loss: 1.9714 - accuracy: 0.3231 - val_loss: 2.1870 - val_accuracy: 0.2857
Epoch 7/50
3/3 [==============================] - 0s 81ms/step - loss: 1.8421 - accuracy: 0.5077 - val_loss: 2.0022 - val_accuracy: 0.4286
Epoch 8/50
3/3 [==========

In [1]:
preds = model.predict(X_test)

NameError: name 'model' is not defined

In [35]:
y_pred = [pred.argmax() for pred in preds]

In [36]:
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         4
           2       0.75      1.00      0.86         3
           3       1.00      1.00      1.00         4
           4       0.43      1.00      0.60         3
           5       0.50      1.00      0.67         1
           6       0.00      0.00      0.00         2
           7       1.00      0.33      0.50         6
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         1

    accuracy                           0.79        28
   macro avg       0.77      0.83      0.76        28
weighted avg       0.82      0.79      0.75        28

0.7857142857142857


In [12]:
y_pred_all = []
y_true_all = []
acertos = 0
for train_index, test_index in tqdm_notebook(loo.split(df['average_image'].values)):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(100,100,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    optimizer = tf.keras.optimizers.Adam()
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = df['average_image'].values[train_index], df['average_image'].values[test_index]
    y_train, y_test = df['label'].values[train_index], df['label'].values[test_index]
    
    X_train = X_train / 255.
    X_test = X_test / 255.
    
    y_test = np.array([dict_map[y_t] for y_t in y_test])

    y_train = np.array([dict_map[y_t] for y_t in y_train])

    X_train = [np.asarray(x).astype(np.float32) for x in X_train]

    X_test = [np.asarray(x).astype(np.float32) for x in X_test]

    X_train = np.array(X_train)

    X_test = np.array(X_test)
    
    model.fit(X_train, y_train,
          epochs=70,
          validation_data=(X_test,y_test))
    
    preds = model.predict(X_test)
    
    y_pred = [pred.argmax() for pred in preds]
    
    y_pred_all.append(y_pred[0])
    y_true_all.append(y_test[0])
    
    if y_pred[0] == y_test[0]:
            acertos += 1
    print(str(acertos)+'/'+str(len(y_pred_all)))

0it [00:00, ?it/s]

Epoch 1/70
3/3 [==============================] - 2s 184ms/step - loss: 2.2772 - accuracy: 0.0870 - val_loss: 2.2242 - val_accuracy: 0.0000e+00
Epoch 2/70
3/3 [==============================] - 0s 80ms/step - loss: 1.9535 - accuracy: 0.4891 - val_loss: 2.0181 - val_accuracy: 0.0000e+00
Epoch 3/70
3/3 [==============================] - 0s 80ms/step - loss: 1.5552 - accuracy: 0.5652 - val_loss: 1.5586 - val_accuracy: 1.0000
Epoch 4/70
3/3 [==============================] - 0s 81ms/step - loss: 1.1290 - accuracy: 0.7065 - val_loss: 0.5186 - val_accuracy: 1.0000
Epoch 5/70
3/3 [==============================] - 0s 80ms/step - loss: 0.8297 - accuracy: 0.7826 - val_loss: 0.2883 - val_accuracy: 1.0000
Epoch 6/70
3/3 [==============================] - 0s 79ms/step - loss: 0.5026 - accuracy: 0.9239 - val_loss: 0.3766 - val_accuracy: 1.0000
Epoch 7/70
3/3 [==============================] - 0s 80ms/step - loss: 0.3799 - accuracy: 0.9022 - val_loss: 0.3645 - val_accuracy: 1.0000
Epoch 8/70
3/3 [==

Epoch 44/70
3/3 [==============================] - 0s 80ms/step - loss: 1.9798e-04 - accuracy: 1.0000 - val_loss: 8.8355e-04 - val_accuracy: 1.0000
Epoch 45/70
3/3 [==============================] - 0s 80ms/step - loss: 1.8663e-04 - accuracy: 1.0000 - val_loss: 8.3733e-04 - val_accuracy: 1.0000
Epoch 46/70
3/3 [==============================] - 0s 82ms/step - loss: 1.7607e-04 - accuracy: 1.0000 - val_loss: 8.0732e-04 - val_accuracy: 1.0000
Epoch 47/70
3/3 [==============================] - 0s 81ms/step - loss: 1.6786e-04 - accuracy: 1.0000 - val_loss: 7.7623e-04 - val_accuracy: 1.0000
Epoch 48/70
3/3 [==============================] - 0s 81ms/step - loss: 1.5910e-04 - accuracy: 1.0000 - val_loss: 7.5312e-04 - val_accuracy: 1.0000
Epoch 49/70
3/3 [==============================] - 0s 80ms/step - loss: 1.4990e-04 - accuracy: 1.0000 - val_loss: 7.3763e-04 - val_accuracy: 1.0000
Epoch 50/70
3/3 [==============================] - 0s 80ms/step - loss: 1.4274e-04 - accuracy: 1.0000 - val_loss

3/3 [==============================] - 0s 87ms/step - loss: 0.0090 - accuracy: 1.0000 - val_loss: 0.0469 - val_accuracy: 1.0000
Epoch 17/70
3/3 [==============================] - 0s 83ms/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 0.0745 - val_accuracy: 1.0000
Epoch 18/70
3/3 [==============================] - 0s 82ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.1156 - val_accuracy: 1.0000
Epoch 19/70
3/3 [==============================] - 0s 82ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.1670 - val_accuracy: 1.0000
Epoch 20/70
3/3 [==============================] - 0s 82ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.2236 - val_accuracy: 1.0000
Epoch 21/70
3/3 [==============================] - 0s 85ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.2712 - val_accuracy: 1.0000
Epoch 22/70
3/3 [==============================] - 0s 84ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.3018 - val_accuracy: 1.0000
Epoch 23/70
3/3 [===============

3/3 [==============================] - 0s 88ms/step - loss: 2.6645e-04 - accuracy: 1.0000 - val_loss: 2.5630e-05 - val_accuracy: 1.0000
Epoch 60/70
3/3 [==============================] - 0s 89ms/step - loss: 2.5866e-04 - accuracy: 1.0000 - val_loss: 2.4914e-05 - val_accuracy: 1.0000
Epoch 61/70
3/3 [==============================] - 0s 86ms/step - loss: 2.5293e-04 - accuracy: 1.0000 - val_loss: 2.3126e-05 - val_accuracy: 1.0000
Epoch 62/70
3/3 [==============================] - 0s 100ms/step - loss: 2.4414e-04 - accuracy: 1.0000 - val_loss: 2.2292e-05 - val_accuracy: 1.0000
Epoch 63/70
3/3 [==============================] - 0s 89ms/step - loss: 2.3847e-04 - accuracy: 1.0000 - val_loss: 2.0504e-05 - val_accuracy: 1.0000
Epoch 64/70
3/3 [==============================] - 0s 86ms/step - loss: 2.3162e-04 - accuracy: 1.0000 - val_loss: 1.9908e-05 - val_accuracy: 1.0000
Epoch 65/70
3/3 [==============================] - 0s 84ms/step - loss: 2.2548e-04 - accuracy: 1.0000 - val_loss: 1.9193e-0

...

3/3 [==============================] - 0s 84ms/step - loss: 3.8971e-04 - accuracy: 1.0000 - val_loss: 0.0056 - val_accuracy: 1.0000
Epoch 49/70
3/3 [==============================] - 0s 89ms/step - loss: 3.8138e-04 - accuracy: 1.0000 - val_loss: 0.0055 - val_accuracy: 1.0000
Epoch 50/70
3/3 [==============================] - 0s 91ms/step - loss: 3.7051e-04 - accuracy: 1.0000 - val_loss: 0.0054 - val_accuracy: 1.0000
Epoch 51/70
3/3 [==============================] - 0s 87ms/step - loss: 3.6025e-04 - accuracy: 1.0000 - val_loss: 0.0053 - val_accuracy: 1.0000
Epoch 52/70
3/3 [==============================] - 0s 87ms/step - loss: 3.4946e-04 - accuracy: 1.0000 - val_loss: 0.0052 - val_accuracy: 1.0000
Epoch 53/70
3/3 [==============================] - 0s 87ms/step - loss: 3.4159e-04 - accuracy: 1.0000 - val_loss: 0.0052 - val_accuracy: 1.0000
Epoch 54/70
3/3 [==============================] - 0s 87ms/step - loss: 3.3485e-04 - accuracy: 1.0000 - val_loss: 0.0051 - val_accuracy: 1.0000
Epoc

Epoch 35/70
3/3 [==============================] - 0s 91ms/step - loss: 5.5743e-04 - accuracy: 1.0000 - val_loss: 8.3446e-07 - val_accuracy: 1.0000
Epoch 36/70
3/3 [==============================] - 0s 89ms/step - loss: 5.2751e-04 - accuracy: 1.0000 - val_loss: 7.1526e-07 - val_accuracy: 1.0000
Epoch 37/70
3/3 [==============================] - 0s 132ms/step - loss: 5.0019e-04 - accuracy: 1.0000 - val_loss: 7.1526e-07 - val_accuracy: 1.0000
Epoch 38/70
3/3 [==============================] - 1s 261ms/step - loss: 4.7522e-04 - accuracy: 1.0000 - val_loss: 5.9605e-07 - val_accuracy: 1.0000
Epoch 39/70
3/3 [==============================] - 1s 221ms/step - loss: 4.5402e-04 - accuracy: 1.0000 - val_loss: 5.9605e-07 - val_accuracy: 1.0000
Epoch 40/70
3/3 [==============================] - 0s 86ms/step - loss: 4.3257e-04 - accuracy: 1.0000 - val_loss: 4.7684e-07 - val_accuracy: 1.0000
Epoch 41/70
3/3 [==============================] - 0s 88ms/step - loss: 4.1275e-04 - accuracy: 1.0000 - val_l

3/3 [==============================] - 0s 87ms/step - loss: 0.3469 - accuracy: 0.9022 - val_loss: 3.8175e-04 - val_accuracy: 1.0000
Epoch 9/70
3/3 [==============================] - 0s 91ms/step - loss: 0.2179 - accuracy: 0.9457 - val_loss: 9.5244e-05 - val_accuracy: 1.0000
Epoch 10/70
3/3 [==============================] - 0s 114ms/step - loss: 0.1881 - accuracy: 0.9674 - val_loss: 5.7219e-05 - val_accuracy: 1.0000
Epoch 11/70
3/3 [==============================] - 1s 264ms/step - loss: 0.0990 - accuracy: 0.9783 - val_loss: 1.4663e-05 - val_accuracy: 1.0000
Epoch 12/70
3/3 [==============================] - 1s 241ms/step - loss: 0.0643 - accuracy: 1.0000 - val_loss: 7.3909e-06 - val_accuracy: 1.0000
Epoch 13/70
3/3 [==============================] - 0s 84ms/step - loss: 0.0398 - accuracy: 1.0000 - val_loss: 3.6955e-06 - val_accuracy: 1.0000
Epoch 14/70
3/3 [==============================] - 0s 87ms/step - loss: 0.0324 - accuracy: 0.9891 - val_loss: 2.3842e-06 - val_accuracy: 1.0000
Ep

Epoch 64/70
3/3 [==============================] - 0s 90ms/step - loss: 2.5322e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 65/70
3/3 [==============================] - 0s 89ms/step - loss: 2.4815e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 66/70
3/3 [==============================] - 0s 142ms/step - loss: 2.4139e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 67/70
3/3 [==============================] - 1s 298ms/step - loss: 2.3766e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 68/70
3/3 [==============================] - 1s 205ms/step - loss: 2.3171e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 69/70
3/3 [==============================] - 0s 84ms/step - loss: 2.2726e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 70/70
3/3 [==============================] - 0s 88ms/step - loss: 2.2260e-04 - accuracy: 1.0000 - val_l

3/3 [==============================] - 0s 90ms/step - loss: 3.5020e-04 - accuracy: 1.0000 - val_loss: 2.0266e-06 - val_accuracy: 1.0000
Epoch 37/70
3/3 [==============================] - 0s 90ms/step - loss: 3.3136e-04 - accuracy: 1.0000 - val_loss: 1.9073e-06 - val_accuracy: 1.0000
Epoch 38/70
3/3 [==============================] - 0s 90ms/step - loss: 3.1976e-04 - accuracy: 1.0000 - val_loss: 1.7881e-06 - val_accuracy: 1.0000
Epoch 39/70
3/3 [==============================] - 0s 90ms/step - loss: 3.0004e-04 - accuracy: 1.0000 - val_loss: 1.7881e-06 - val_accuracy: 1.0000
Epoch 40/70
3/3 [==============================] - 0s 89ms/step - loss: 2.8319e-04 - accuracy: 1.0000 - val_loss: 1.9073e-06 - val_accuracy: 1.0000
Epoch 41/70
3/3 [==============================] - 0s 91ms/step - loss: 2.7086e-04 - accuracy: 1.0000 - val_loss: 1.7881e-06 - val_accuracy: 1.0000
Epoch 42/70
3/3 [==============================] - 0s 90ms/step - loss: 2.5176e-04 - accuracy: 1.0000 - val_loss: 1.5497e-06

In [13]:
print(classification_report(y_true_all, y_pred_all))
print(accuracy_score(y_true_all, y_pred_all))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      1.00      1.00         9
           2       0.70      0.78      0.74         9
           3       0.90      1.00      0.95         9
           4       0.88      0.70      0.78        10
           5       1.00      1.00      1.00         9
           6       0.75      0.60      0.67        10
           7       0.83      1.00      0.91        10
           8       0.89      0.89      0.89         9
           9       1.00      1.00      1.00         9

    accuracy                           0.89        93
   macro avg       0.89      0.90      0.89        93
weighted avg       0.89      0.89      0.89        93

0.8924731182795699


# RNN

In [6]:

# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

In [7]:
class SpatialAttention(tf.keras.layers.Layer):
      def __init__(self, kernel_size):
        super(SpatialAttention, self).__init__()
        self.kernel_size = kernel_size
        
        def build(self, input_shape):
            self.conv2d = tf.keras.layers.Conv2D(filters = 1,
                    kernel_size=self.kernel_size,
                    strides=1,
                    padding='same',
                    activation='sigmoid',
                    kernel_initializer='he_normal',
                    use_bias=False)

        def call(self, inputs):
            
            # AvgPool
            avg_pool = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=3, keepdims=True))(inputs)
            
            # MaxPool
            max_pool = tf.keras.layers.Lambda(lambda x: tf.keras.backend.max(x, axis=3, keepdims=True))(inputs)

            attention = tf.keras.layers.Concatenate(axis=3)([avg_pool, max_pool])

            attention = self.conv2d(attention)


            return tf.keras.layers.multiply([inputs, attention]) 

In [8]:
attention_layer = attention()

In [9]:
def create_LRCN_model():
    '''
    This function will construct the required LRCN model.
    Returns:
        model: It is the required constructed LRCN model.
    '''

    # We will use a Sequential model for model construction.
    
    model = Sequential()    
    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same',activation = 'relu'),
                              input_shape = (29, 160, 160, 3)))
    model.add(TimeDistributed(MaxPooling2D((3, 3)))) 
    model.add(TimeDistributed(Dropout(0.25)))
    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((3, 3))))
#     model.add(SpatialAttention(7))
    model.add(TimeDistributed(Dropout(0.25))) 
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.25)))
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
#     model.add(SpatialAttention(7))
    model.add(TimeDistributed(Flatten()))           
#     model.add(LSTM(32, return_sequences=True))
#     model.add(attention_layer)
#     model.add(Dense(10, trainable=True, activation = 'softmax'))
    model.add(LSTM(32))
    model.add(Dense(10, trainable=True, activation = 'softmax'))

   
    # Return the constructed LRCN model.
    return model

In [10]:
dict_data = {'label': [], 'features': []}
counts = []
for skeleton_folder in tqdm_notebook(skeletons_action_folders):
    
    current_action = skeleton_folder.split('\\')[1]
    
    actors_folders = glob(skeleton_folder.replace('\\', '/')+'/*')
    
    for actor_folder in actors_folders:
        
        images = glob(actor_folder.replace('\\', '/')+'/*')
        
        if len(images) > 10:
        
            images.sort(key=len)

            dict_data['label'].append(current_action)

            w,h=Image.open(images[0].replace('\\', '/')).size
            N=len(images)

            # Create a numpy array of floats to store the average (assume RGB images)
            arr=np.zeros((h,w,3))

            feats = []

            # Build up average pixel intensities, casting each image as an array of floats
            for im in images[:29]:
                # Ver o vídeo com maior número de frames e aplicar o padding aqui para garantir que todos tenham a mesma
                # dimensão
                feats.append(np.array(Image.open(im)))
                
            for i in np.arange(len(feats), 29):
                a = np.zeros(shape=(160, 160,3), dtype='uint8')
                feats.append(a)

            # Round values in array and cast as 8-bit integer


            dict_data['features'].append(feats)
        counts.append(len(images))

  0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
len(dict_data)

2

In [12]:
dict_data['features'][0][0].shape

(160, 160, 3)

In [13]:
df = pd.DataFrame(dict_data)

In [14]:
 len(dict_data['features'][1])

29

In [15]:
len(df)

93

In [16]:
Counter(df['label'])

Counter({'bend': 9,
         'jack': 9,
         'jump': 9,
         'pjump': 9,
         'run': 10,
         'side': 9,
         'skip': 10,
         'walk': 10,
         'wave1': 9,
         'wave2': 9})

In [17]:
df['features'] = df['features'].apply(lambda x: np.asarray(x).astype(np.float32))

In [20]:
y = pd.get_dummies(df.label, prefix='')

In [20]:
y = y.apply(lambda x: np.asarray(x).astype(np.float32))

In [21]:
y.values[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [22]:
features_train, features_test, labels_train, labels_test = train_test_split(df['features'].values, y.values,
                                                                            test_size = 0.3,
                                                                            random_state = 42)

In [23]:
features_train = np.asarray(list(features_train)).astype(np.float32)

In [24]:
features_test = np.asarray(list(features_test)).astype(np.float32)

In [25]:
features_train = features_train /255

In [26]:
features_test = features_test / 255

In [32]:
model = create_LRCN_model()
# model = create_convlstm_model()

In [33]:
# Create an Instance of Early Stopping Callback.
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 20, mode = 'min', restore_best_weights = True)
 
# Compile the model and specify loss function, optimizer and metrics to the model.
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

In [34]:
# Start training the model.
model.fit(x = features_train, y = labels_train, epochs = 70, batch_size = 8, validation_data=(features_test, np.array(labels_test)), callbacks = [early_stopping_callback])

Epoch 1/70
9/9 [==============================] - 16s 176ms/step - loss: 2.4025 - accuracy: 0.0769 - val_loss: 2.3482 - val_accuracy: 0.0357
Epoch 2/70
9/9 [==============================] - 1s 73ms/step - loss: 2.2800 - accuracy: 0.1077 - val_loss: 2.3126 - val_accuracy: 0.0714
Epoch 3/70
9/9 [==============================] - 1s 72ms/step - loss: 2.2868 - accuracy: 0.1231 - val_loss: 2.3140 - val_accuracy: 0.1071
Epoch 4/70
9/9 [==============================] - 1s 72ms/step - loss: 2.2726 - accuracy: 0.1846 - val_loss: 2.3097 - val_accuracy: 0.1071
Epoch 5/70
9/9 [==============================] - 1s 72ms/step - loss: 2.2542 - accuracy: 0.2154 - val_loss: 2.2896 - val_accuracy: 0.2500
Epoch 6/70
9/9 [==============================] - 1s 73ms/step - loss: 2.2062 - accuracy: 0.2462 - val_loss: 2.2321 - val_accuracy: 0.1786
Epoch 7/70
9/9 [==============================] - 1s 73ms/step - loss: 2.0829 - accuracy: 0.3077 - val_loss: 2.1248 - val_accuracy: 0.1786
Epoch 8/70
9/9 [=========

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Anaconda\envs\harbop\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-34-e92d5b552c5d>", line 2, in <module>
    model.fit(x = features_train, y = labels_train, epochs = 70, batch_size = 8, validation_data=(features_test, np.array(labels_test)), callbacks = [early_stopping_callback])
  File "C:\Anaconda\envs\harbop\lib\site-packages\keras\engine\training.py", line 1226, in fit
    _use_cached_eval_dataset=True)
  File "C:\Anaconda\envs\harbop\lib\site-packages\keras\engine\training.py", line 1501, in evaluate
    tmp_logs = self.test_function(iterator)
  File "C:\Anaconda\envs\harbop\lib\site-packages\tensorflow\python\eager\def_function.py", line 885, in __call__
    result = self._call(*args, **kwds)
  File "C:\Anaconda\envs\harbop\lib\site-packages\tensorflow\python\eager\def_function.py", line 924, in _call
    results = self._statefu

TypeError: object of type 'NoneType' has no len()

In [58]:
preds = model.predict(features_test)

In [59]:
y_pred = [pred.argmax() for pred in preds]

In [60]:
y_pred

[4,
 2,
 6,
 4,
 0,
 2,
 4,
 7,
 1,
 4,
 8,
 3,
 9,
 2,
 1,
 0,
 6,
 8,
 5,
 6,
 1,
 6,
 8,
 3,
 3,
 1,
 4,
 4]

In [61]:
y_true = []
for vect in labels_test:
    for i, value in enumerate(vect):
        if value == 1:
            
            y_true.append(i)

In [62]:
y_true

[4,
 2,
 6,
 7,
 0,
 2,
 4,
 7,
 1,
 4,
 8,
 3,
 9,
 6,
 1,
 0,
 2,
 3,
 5,
 7,
 1,
 7,
 8,
 3,
 3,
 1,
 7,
 7]

In [63]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         4
           2       0.67      0.67      0.67         3
           3       1.00      0.75      0.86         4
           4       0.50      1.00      0.67         3
           5       1.00      1.00      1.00         1
           6       0.25      0.50      0.33         2
           7       1.00      0.17      0.29         6
           8       0.67      1.00      0.80         2
           9       1.00      1.00      1.00         1

    accuracy                           0.71        28
   macro avg       0.81      0.81      0.76        28
weighted avg       0.83      0.71      0.69        28



In [64]:
print(accuracy_score(y_true, y_pred))

0.7142857142857143


In [93]:
y_true_all = pickle.load(open('y_true_all.pkl', 'rb'))
y_pred_all = pickle.load(open('y_pred_all.pkl', 'rb'))

In [13]:
len(y_true_all)

33

In [29]:
temp = []
checkpoint = 0

for train_index, test_index in tqdm_notebook(loo.split(df['features'].values)):
    if checkpoint >= 0:
        y = pd.get_dummies(df['label'], prefix='')

        y = y.apply(lambda x: np.asarray(x).astype(np.float32))
        labels_train, labels_test = y.values[train_index], y.values[test_index]
        y_true = []
        for vect in labels_test:
            for i, value in enumerate(vect):
                if value == 1:

                    y_true.append(i)
        temp.append(y_true[0])
    checkpoint += 1

0it [00:00, ?it/s]

In [19]:
print(y_true_all)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3]


In [18]:
print(y_pred_all)

[0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 6, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 9]


In [30]:
print(temp)

[3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9]


In [18]:
loo = LeaveOneOut()

In [19]:
y_pred_all = []
y_true_all = []
acertos = 0
checkpoint = 0
for train_index, test_index in tqdm_notebook(loo.split(df['features'].values)):
    print(checkpoint)
    if checkpoint >= 0:
        
        model = create_LRCN_model()

        # Create an Instance of Early Stopping Callback.
        early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 20, mode = 'min', restore_best_weights = True)

        # Compile the model and specify loss function, optimizer and metrics to the model.
        model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

        y = pd.get_dummies(df['label'], prefix='')

        y = y.apply(lambda x: np.asarray(x).astype(np.float32))

        features_train, features_test = df['features'].values[train_index], df['features'].values[test_index]
        labels_train, labels_test = y.values[train_index], y.values[test_index]

        features_train = np.asarray(list(features_train)).astype(np.float32)

        features_test = np.asarray(list(features_test)).astype(np.float32)

        features_train = features_train / 255

        features_test = features_test / 255

        # Start training the model.
        model.fit(x = features_train, y = labels_train, epochs = 70, batch_size = 8, validation_data=(features_test, np.array(labels_test)), callbacks=[early_stopping_callback])

        preds = model.predict(features_test)

        y_pred = [pred.argmax() for pred in preds]

        y_pred_all.append(y_pred[0])
        y_true = []
        for vect in labels_test:
            for i, value in enumerate(vect):
                if value == 1:

                    y_true.append(i)
        y_true_all.append(y_true[0])

        if y_pred[0] == y_true[0]:
            acertos += 1
        print(str(acertos)+'/'+str(len(y_pred_all)))
        
    checkpoint += 1

0it [00:00, ?it/s]

0
Epoch 1/70
12/12 [==============================] - 79s 239ms/step - loss: 2.4253 - accuracy: 0.0761 - val_loss: 2.2862 - val_accuracy: 0.0000e+00
Epoch 2/70
12/12 [==============================] - 2s 141ms/step - loss: 2.3173 - accuracy: 0.0435 - val_loss: 2.3264 - val_accuracy: 0.0000e+00
Epoch 3/70
12/12 [==============================] - 2s 140ms/step - loss: 2.3051 - accuracy: 0.0761 - val_loss: 2.3039 - val_accuracy: 0.0000e+00
Epoch 4/70
12/12 [==============================] - 2s 140ms/step - loss: 2.3003 - accuracy: 0.1087 - val_loss: 2.3181 - val_accuracy: 0.0000e+00
Epoch 5/70
12/12 [==============================] - 2s 142ms/step - loss: 2.2990 - accuracy: 0.0761 - val_loss: 2.3488 - val_accuracy: 0.0000e+00
Epoch 6/70
12/12 [==============================] - 2s 143ms/step - loss: 2.2959 - accuracy: 0.1413 - val_loss: 2.3667 - val_accuracy: 0.0000e+00
Epoch 7/70
12/12 [==============================] - 2s 141ms/step - loss: 2.2888 - accuracy: 0.1196 - val_loss: 2.3211 - 

12/12 [==============================] - 2s 142ms/step - loss: 0.0193 - accuracy: 1.0000 - val_loss: 0.0103 - val_accuracy: 1.0000
Epoch 59/70
12/12 [==============================] - 2s 141ms/step - loss: 0.0186 - accuracy: 1.0000 - val_loss: 0.0142 - val_accuracy: 1.0000
Epoch 60/70
12/12 [==============================] - 2s 141ms/step - loss: 0.0182 - accuracy: 1.0000 - val_loss: 0.0100 - val_accuracy: 1.0000
Epoch 61/70
12/12 [==============================] - 2s 142ms/step - loss: 0.0183 - accuracy: 1.0000 - val_loss: 0.0095 - val_accuracy: 1.0000
Epoch 62/70
12/12 [==============================] - 2s 143ms/step - loss: 0.0165 - accuracy: 1.0000 - val_loss: 0.0100 - val_accuracy: 1.0000
Epoch 63/70
12/12 [==============================] - 2s 143ms/step - loss: 0.0162 - accuracy: 1.0000 - val_loss: 0.0143 - val_accuracy: 1.0000
Epoch 64/70
12/12 [==============================] - 2s 142ms/step - loss: 0.0152 - accuracy: 1.0000 - val_loss: 0.0089 - val_accuracy: 1.0000
Epoch 65/70

12/12 [==============================] - 2s 143ms/step - loss: 0.0887 - accuracy: 0.9891 - val_loss: 0.0116 - val_accuracy: 1.0000
Epoch 46/70
12/12 [==============================] - 2s 143ms/step - loss: 0.0638 - accuracy: 1.0000 - val_loss: 0.0092 - val_accuracy: 1.0000
Epoch 47/70
12/12 [==============================] - 2s 143ms/step - loss: 0.0517 - accuracy: 1.0000 - val_loss: 0.0087 - val_accuracy: 1.0000
Epoch 48/70
12/12 [==============================] - 2s 143ms/step - loss: 0.0453 - accuracy: 1.0000 - val_loss: 0.0083 - val_accuracy: 1.0000
Epoch 49/70
12/12 [==============================] - 2s 143ms/step - loss: 0.0433 - accuracy: 1.0000 - val_loss: 0.0081 - val_accuracy: 1.0000
Epoch 50/70
12/12 [==============================] - 2s 142ms/step - loss: 0.0415 - accuracy: 1.0000 - val_loss: 0.0078 - val_accuracy: 1.0000
Epoch 51/70
12/12 [==============================] - 2s 144ms/step - loss: 0.0400 - accuracy: 1.0000 - val_loss: 0.0076 - val_accuracy: 1.0000
Epoch 52/70

12/12 [==============================] - 2s 154ms/step - loss: 0.9511 - accuracy: 0.7826 - val_loss: 0.1362 - val_accuracy: 1.0000
Epoch 33/70
12/12 [==============================] - 2s 155ms/step - loss: 0.9220 - accuracy: 0.8152 - val_loss: 0.2410 - val_accuracy: 1.0000
Epoch 34/70
12/12 [==============================] - 2s 155ms/step - loss: 1.0801 - accuracy: 0.6413 - val_loss: 0.3674 - val_accuracy: 1.0000
Epoch 35/70
12/12 [==============================] - 2s 154ms/step - loss: 1.0388 - accuracy: 0.6957 - val_loss: 0.4269 - val_accuracy: 1.0000
Epoch 36/70
12/12 [==============================] - 2s 155ms/step - loss: 0.9644 - accuracy: 0.7826 - val_loss: 0.3712 - val_accuracy: 1.0000
Epoch 37/70
12/12 [==============================] - 2s 154ms/step - loss: 0.9380 - accuracy: 0.7500 - val_loss: 0.1806 - val_accuracy: 1.0000
Epoch 38/70
12/12 [==============================] - 2s 154ms/step - loss: 0.8621 - accuracy: 0.8370 - val_loss: 0.1623 - val_accuracy: 1.0000
Epoch 39/70

...

Epoch 21/70
12/12 [==============================] - 9s 713ms/step - loss: 1.6930 - accuracy: 0.4022 - val_loss: 2.1001 - val_accuracy: 0.0000e+00
Epoch 22/70
12/12 [==============================] - 9s 779ms/step - loss: 1.6629 - accuracy: 0.4130 - val_loss: 1.5837 - val_accuracy: 0.0000e+00
Epoch 23/70
12/12 [==============================] - 9s 807ms/step - loss: 1.8146 - accuracy: 0.3261 - val_loss: 2.5232 - val_accuracy: 0.0000e+00
Epoch 24/70
12/12 [==============================] - 6s 475ms/step - loss: 1.4741 - accuracy: 0.5109 - val_loss: 2.9283 - val_accuracy: 0.0000e+00
Epoch 25/70
12/12 [==============================] - 9s 814ms/step - loss: 1.4453 - accuracy: 0.5217 - val_loss: 2.0629 - val_accuracy: 0.0000e+00
Epoch 26/70
12/12 [==============================] - 13s 1s/step - loss: 1.2406 - accuracy: 0.5652 - val_loss: 1.6841 - val_accuracy: 0.0000e+00
Epoch 27/70
12/12 [==============================] - 18s 1s/step - loss: 1.1716 - accuracy: 0.6304 - val_loss: 1.6842 - 

12/12 [==============================] - 3s 239ms/step - loss: 0.1529 - accuracy: 1.0000 - val_loss: 0.1122 - val_accuracy: 1.0000
Epoch 36/70
12/12 [==============================] - 3s 237ms/step - loss: 0.1295 - accuracy: 1.0000 - val_loss: 0.1191 - val_accuracy: 1.0000
Epoch 37/70
12/12 [==============================] - 3s 243ms/step - loss: 0.1151 - accuracy: 1.0000 - val_loss: 0.0891 - val_accuracy: 1.0000
Epoch 38/70
12/12 [==============================] - 3s 260ms/step - loss: 0.1003 - accuracy: 1.0000 - val_loss: 0.0776 - val_accuracy: 1.0000
Epoch 39/70
12/12 [==============================] - 4s 273ms/step - loss: 0.0931 - accuracy: 1.0000 - val_loss: 0.0735 - val_accuracy: 1.0000
Epoch 40/70
12/12 [==============================] - 6s 555ms/step - loss: 0.0999 - accuracy: 0.9891 - val_loss: 0.0781 - val_accuracy: 1.0000
Epoch 41/70
12/12 [==============================] - 2s 188ms/step - loss: 0.0949 - accuracy: 0.9891 - val_loss: 0.0723 - val_accuracy: 1.0000
Epoch 42/70

Epoch 22/70
12/12 [==============================] - 8s 681ms/step - loss: 0.3535 - accuracy: 0.9348 - val_loss: 0.4809 - val_accuracy: 1.0000
Epoch 23/70
12/12 [==============================] - 3s 299ms/step - loss: 0.3019 - accuracy: 0.9891 - val_loss: 0.0929 - val_accuracy: 1.0000
Epoch 24/70
12/12 [==============================] - 9s 804ms/step - loss: 0.2701 - accuracy: 0.9891 - val_loss: 0.1470 - val_accuracy: 1.0000
Epoch 25/70
12/12 [==============================] - 5s 393ms/step - loss: 0.2575 - accuracy: 0.9783 - val_loss: 0.0764 - val_accuracy: 1.0000
Epoch 26/70
12/12 [==============================] - 4s 389ms/step - loss: 0.2299 - accuracy: 0.9783 - val_loss: 0.0353 - val_accuracy: 1.0000
Epoch 27/70
12/12 [==============================] - 6s 518ms/step - loss: 0.2159 - accuracy: 0.9674 - val_loss: 0.0581 - val_accuracy: 1.0000
Epoch 28/70
12/12 [==============================] - 2s 204ms/step - loss: 0.1858 - accuracy: 0.9891 - val_loss: 0.0533 - val_accuracy: 1.0000

12/12 [==============================] - 4s 347ms/step - loss: 1.7589 - accuracy: 0.4022 - val_loss: 2.1745 - val_accuracy: 0.0000e+00
Epoch 10/70
12/12 [==============================] - 4s 347ms/step - loss: 1.8307 - accuracy: 0.3043 - val_loss: 1.9723 - val_accuracy: 0.0000e+00
Epoch 11/70
12/12 [==============================] - 2s 203ms/step - loss: 1.6446 - accuracy: 0.3696 - val_loss: 2.0276 - val_accuracy: 0.0000e+00
Epoch 12/70
12/12 [==============================] - 3s 261ms/step - loss: 1.4506 - accuracy: 0.4239 - val_loss: 1.2714 - val_accuracy: 1.0000
Epoch 13/70
12/12 [==============================] - 5s 407ms/step - loss: 1.3159 - accuracy: 0.5000 - val_loss: 1.4129 - val_accuracy: 0.0000e+00
Epoch 14/70
12/12 [==============================] - 8s 646ms/step - loss: 1.2455 - accuracy: 0.5326 - val_loss: 1.1423 - val_accuracy: 1.0000
Epoch 15/70
12/12 [==============================] - 8s 678ms/step - loss: 1.1628 - accuracy: 0.5870 - val_loss: 1.0132 - val_accuracy: 1.

12/12 [==============================] - 4s 338ms/step - loss: 0.0246 - accuracy: 1.0000 - val_loss: 0.0110 - val_accuracy: 1.0000
Epoch 67/70
12/12 [==============================] - 2s 198ms/step - loss: 0.0250 - accuracy: 1.0000 - val_loss: 0.0110 - val_accuracy: 1.0000
Epoch 68/70
12/12 [==============================] - 3s 217ms/step - loss: 0.0231 - accuracy: 1.0000 - val_loss: 0.0108 - val_accuracy: 1.0000
Epoch 69/70
12/12 [==============================] - 3s 212ms/step - loss: 0.0232 - accuracy: 1.0000 - val_loss: 0.0105 - val_accuracy: 1.0000
Epoch 70/70
12/12 [==============================] - 3s 221ms/step - loss: 0.0224 - accuracy: 1.0000 - val_loss: 0.0102 - val_accuracy: 1.0000
84/93


In [20]:
y_true = []
for vect in labels_test:
    for i, value in enumerate(vect):
        if value == 1:

            y_true.append(i)

In [21]:
print(accuracy_score(y_true_all, y_pred_all))

0.9032258064516129


In [51]:
print(classification_report(y_true_all, y_pred_all))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      1.00      1.00         9
           2       0.82      1.00      0.90         9
           3       0.90      1.00      0.95         9
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00         9
           6       1.00      0.70      0.82        10
           7       0.91      1.00      0.95        10
           8       1.00      0.89      0.94         9
           9       1.00      1.00      1.00         9

    accuracy                           0.96        93
   macro avg       0.96      0.96      0.96        93
weighted avg       0.96      0.96      0.96        93



In [52]:
pickle.dump(y_true_all, open('y_true_all_normal-same_color.pkl', 'wb'))
pickle.dump(y_pred_all, open('y_pred_all_normal-same_color.pkl', 'wb'))

In [6]:
y_true_1 = pickle.load(open('y_true_all_0-77_pif2.pkl', 'rb'))
y_pred_1 = pickle.load(open('y_pred_all_0-77_pif2.pkl', 'rb'))

In [7]:
y_true_2 = pickle.load(open('y_true_all_77-93_pif2.pkl', 'rb'))
y_pred_2 = pickle.load(open('y_pred_all_77-93_pif2.pkl', 'rb'))

In [9]:
y_true_1.extend(y_true_2)
# y_true_1.extend(y_true_all)

In [10]:
len(y_true_1)

93

In [11]:
# y_pred_1.extend(y_pred_2)
y_pred_1.extend(y_pred_2)

In [12]:
print(len(y_pred_1))
print(len(y_true_1))

93
93


In [13]:
print(classification_report(y_true_1, y_pred_1))

              precision    recall  f1-score   support

           0       1.00      0.89      0.94         9
           1       1.00      1.00      1.00         9
           2       0.67      0.67      0.67         9
           3       0.82      1.00      0.90         9
           4       0.83      0.50      0.62        10
           5       0.78      0.78      0.78         9
           6       0.38      0.50      0.43        10
           7       0.82      0.90      0.86        10
           8       1.00      0.89      0.94         9
           9       1.00      1.00      1.00         9

    accuracy                           0.81        93
   macro avg       0.83      0.81      0.81        93
weighted avg       0.82      0.81      0.81        93



In [14]:
accuracy_score(y_true_1, y_pred_1)

0.8064516129032258

# Coordenadas PifPaf

In [45]:
skeletons_action_folders = glob('./datasets/Weizmann/coordinatesfeatures/*')

In [51]:
dict_data = {'label': [], 'coordinates': []}
for skeleton_folder in tqdm_notebook(skeletons_action_folders):
    
    current_action = skeleton_folder.split('\\')[1]
    
    actors_folders = glob(skeleton_folder.replace('\\', '/')+'/*')
    
    for actor_folder in actors_folders:
        
        dict_data['label'].append(current_action)
        
        coordinates = glob(actor_folder.replace('\\', '/')+'/*')
        
        temp_coor = []
        for coordinate in coordinates[:29]:
            
            values = pickle.load(open(coordinate, "rb"))
            
            new_values = np.array([x_y[:2] for x_y in values])
            
            temp_coor.append(new_values)
        
        dict_data['coordinates'].append(temp_coor)
        # Generate, save and preview final image

  0%|          | 0/10 [00:00<?, ?it/s]

In [53]:
dict_data['coordinates'][2][0].shape

(13, 2)

In [74]:
model = Sequential()    
model.add(TimeDistributed(Conv1D(16, (3), padding='same',activation = 'relu'),
                          input_shape = (29, 13, 2)))
model.add(TimeDistributed(MaxPooling1D((3), padding='same'))) 
model.add(TimeDistributed(Dropout(0.25)))
model.add(TimeDistributed(Conv1D(32, (3), padding='same',activation = 'relu')))
model.add(TimeDistributed(MaxPooling1D((3), padding='same')))
model.add(TimeDistributed(Dropout(0.25))) 
model.add(TimeDistributed(Conv1D(64, (3), padding='same',activation = 'relu')))
model.add(TimeDistributed(MaxPooling1D((2), padding='same')))
model.add(TimeDistributed(Dropout(0.25)))
model.add(TimeDistributed(Conv1D(64, (3), padding='same',activation = 'relu')))
model.add(TimeDistributed(MaxPooling1D((2), padding='same')))
model.add(TimeDistributed(Flatten()))           
model.add(LSTM(32))
# attention_layer = attention()
model.add(Dense(10, activation = 'softmax'))

In [26]:
[x_y[:2] for x_y in values]

[array([66.11334 , 48.261112], dtype=float32),
 array([73.48276 , 53.668118], dtype=float32),
 array([61.1155  , 54.654747], dtype=float32),
 array([81.16612, 47.81559], dtype=float32),
 array([53.003643, 49.764084], dtype=float32),
 array([85.72636 , 39.207138], dtype=float32),
 array([47.043648, 42.38383 ], dtype=float32),
 array([71.62008, 77.34625], dtype=float32),
 array([63.71472, 77.16902], dtype=float32),
 array([72.34654 , 94.519844], dtype=float32),
 array([64.64294, 94.27105], dtype=float32),
 array([ 73.996765, 110.23986 ], dtype=float32),
 array([ 66.031456, 109.85799 ], dtype=float32)]